<a href="https://colab.research.google.com/github/Shefatmim01/Used-Car-Price-Prediction/blob/main/Used_Car_Price_Prediction(final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Used Car Price Predection**

 **Import necessary libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from google.colab import drive

 **Load Dataset**

In [2]:
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Australian Vehicle Prices.csv'
data = pd.read_csv(file_path)

Mounted at /content/drive


In [ ]:
data.head()

In [ ]:
data.tail()

### **Data Preprocessing**

In [ ]:
# data = data.drop(columns=["Unnamed: 0"])

**Missing Values Handling**

In [ ]:
print('Number of instances = %d' % (data.shape[0]))
print('Number of attributes = %d' % (data.shape[1]))

print('Number of missing values:')
for col in data.columns:
    print('\t%s: %d' % (col,data[col].isna().sum()))

**Delete missing values in numerical columns**

In [ ]:
data=data.dropna()
data.isnull().sum()

In [ ]:
print(data.isnull().sum())

In [ ]:
data.shape

 **Duplicate Data Handeling**

In [ ]:
dups = data.duplicated()
print('Number of duplicate rows = %d' % (dups.sum()))

**Lebel Encoding**

In [ ]:
print(data.dtypes)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder for each categorical column
label_encoders = {}
categorical_columns = data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Store encoder for future use

# Display the updated dataset with encoded categorical columns
data.dtypes


In [ ]:
# converting 'Year'  float to int
data['Year'] = data['Year'].apply(np.int64)

data.dtypes

**EDA**

In [ ]:
data.describe()

**Quartile**

In [ ]:
#creating dataset
np.random.seed(10)
Data=data["Price"]
print(Data)
fig = plt.figure(figsize=(5, 7))

#creating plot
plt.boxplot(Data)

#show plot
plt.show()

 **Calculate The Correlation**

In [ ]:
data.corr()

In [ ]:
plt.figure(figsize=(25,10))
sns.heatmap(data.corr(), annot=True, fmt=".2f")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
print(sns.__version__)

In [ ]:
sns.pairplot(data, height=1.5);
plt.show()

**Split Data into Training and Testing Sets**

In [ ]:
# Data split
X = data.drop(columns=['Price','Brand','Car/Suv'])
Y = data['Price']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=2)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the training data, and only transform the test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled data back to DataFrame (optional, for easier interpretation)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Verify the scaling (mean should be close to 0, std should be close to 1 for X_train)
print("Training data after scaling:")
print(X_train_scaled.describe())


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
min_max_scaler = MinMaxScaler()

# Fit and transform the training data, and only transform the test data
X_train_minmax_scaled = min_max_scaler.fit_transform(X_train)
X_test_minmax_scaled = min_max_scaler.transform(X_test)

# Convert scaled data back to DataFrame (optional, for easier interpretation)
X_train_minmax_scaled = pd.DataFrame(X_train_minmax_scaled, columns=X_train.columns)
X_test_minmax_scaled = pd.DataFrame(X_test_minmax_scaled, columns=X_test.columns)

# Verify the scaling (values should now be between 0 and 1)
print("Training data after MinMax scaling:")
print(X_train_minmax_scaled.describe())


**Define a function for model evaluation**


 **linear regression**

In [ ]:
# Creating and training the Linear Regression model
model = LinearRegression()
model.fit(X_train, Y_train)

Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

In [ ]:
print("Linear Regression Model\n")

# Evaluation Metrics for Training Data
train_mae = mean_absolute_error(Y_train, Y_train_pred)
train_mse = mean_squared_error(Y_train, Y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(Y_train, Y_train_pred)

# Printing the results
print("\tTraining Data Evaluation:")
print(f"\t\tMAE: {train_mae}")
print(f"\t\tMSE: {train_mse}")
print(f"\t\tRMSE: {train_rmse}")
print(f"\t\tR²: {train_r2*100}")


# Evaluation Metrics for Testing Data
test_mae = mean_absolute_error(Y_test, Y_test_pred)
test_mse = mean_squared_error(Y_test, Y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(Y_test, Y_test_pred)

# Printing the results
print("\n\tTesting Data Evaluation:")
print(f"\t\tMAE: {test_mae}")
print(f"\t\tMSE: {test_mse}")
print(f"\t\tRMSE: {test_rmse}")
print(f"\t\tR²: {test_r2*100}")

**Visualize the actual prices and Predicted prices**

In [ ]:
plt.figure(figsize=(8, 4))

# Scatter plot for Training Data
plt.subplot(1, 2, 1)
plt.scatter(Y_train, Y_train_pred, color='blue', alpha=0.6)
plt.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTraining Data(Linear Regression Model)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Scatter plot for Testing Data
plt.subplot(1, 2, 2)
plt.scatter(Y_test, Y_test_pred, color='green', alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTesting Data(Linear Regression Model)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Show the plots
plt.tight_layout()
plt.show()

**Lasso Regression**

In [ ]:
from sklearn.linear_model import Lasso

# loading the linear regression model
lass_reg_model = Lasso()
lass_reg_model.fit(X_train,Y_train)

In [ ]:
# Evaluation Metrics for Training Data
train_mae = mean_absolute_error(Y_train, Y_train_pred)
train_mse = mean_squared_error(Y_train, Y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(Y_train, Y_train_pred)
print("Lasso Regression -\n")
print("\tTraining Data Evaluation:")
print(f"\t\tMAE: {train_mae}")
print(f"\t\tMSE: {train_mse}")
print(f"\t\tRMSE: {train_rmse}")
print(f"\t\tR²: {train_r2*100}")

# Evaluation Metrics for Testing Data
test_mae = mean_absolute_error(Y_test, Y_test_pred)
test_mse = mean_squared_error(Y_test, Y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(Y_test, Y_test_pred)

print("\n\tTesting Data Evaluation:")
print(f"\t\tMAE: {test_mae}")
print(f"\t\tMSE: {test_mse}")
print(f"\t\tRMSE: {test_rmse}")
print(f"\t\tR²: {test_r2*100}")

**Visualize the actual prices and Predicted prices**

In [ ]:
plt.figure(figsize=(8, 4))

# Plotting Actual vs Predicted for Training Data
plt.subplot(1, 2, 1)
plt.scatter(Y_train, Y_train_pred, color='blue', alpha=0.6)
plt.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTraining Data (Lasso Regression)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Plotting Actual vs Predicted for Testing Data
plt.subplot(1, 2, 2)
plt.scatter(Y_test, Y_test_pred, color='green', alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTesting Data (Lasso Regression)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Display the plots
plt.tight_layout()
plt.show()


**Random Forest Regression**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Creating and training the Random Forest Regression model
rf = RandomForestRegressor()
rf.fit(X_train, Y_train)

# Making predictions
Y_train_pred = rf.predict(X_train)
Y_test_pred = rf.predict(X_test)

In [ ]:
# Evaluation Metrics for Training Data
train_mae = mean_absolute_error(Y_train, Y_train_pred)
train_mse = mean_squared_error(Y_train, Y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(Y_train, Y_train_pred)


print("Random Forest Regression\n")
# Printing the results
print("\tTraining Data Evaluation:")
print(f"\t\tMAE: {train_mae}")
print(f"\t\tMSE: {train_mse}")
print(f"\t\tRMSE: {train_rmse}")
print(f"\t\tR²: {train_r2*100}")

# Evaluation Metrics for Testing Data
test_mae = mean_absolute_error(Y_test, Y_test_pred)
test_mse = mean_squared_error(Y_test, Y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(Y_test, Y_test_pred)

print("\n\tTesting Data Evaluation:")
print(f"\t\tMAE: {test_mae}")
print(f"\t\tMSE: {test_mse}")
print(f"\t\tRMSE: {test_rmse}")
print(f"\t\tR²: {test_r2*100}")

**Visualize the actual prices and Predicted prices**

In [ ]:
# Plotting Actual vs Predicted for Training Data
plt.figure(figsize=(8,4))
plt.subplot(1, 2, 1)
plt.scatter(Y_train, Y_train_pred, color='blue', alpha=0.6)
plt.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTraining Data (Random Forest)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Plotting Actual vs Predicted for Testing Data
plt.subplot(1, 2, 2)
plt.scatter(Y_test, Y_test_pred, color='green', alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTesting Data (Random Forest)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Display the plots
plt.tight_layout()
plt.show()

 **XGBoost Regression**

In [ ]:
from xgboost import XGBRegressor

# Creating and training the XGBoost Regression model
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=1)
xgb.fit(X_train, Y_train)

# Making predictions
Y_train_pred = xgb.predict(X_train)
Y_test_pred = xgb.predict(X_test)

In [ ]:
# Evaluation Metrics for Training Data
train_mae = mean_absolute_error(Y_train, Y_train_pred)
train_mse = mean_squared_error(Y_train, Y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(Y_train, Y_train_pred)

print("XGBoost Regression-\n")

# Printing the results
print("\tTraining Data Evaluation:")
print(f"\t\tMAE: {train_mae}")
print(f"\t\tMSE: {train_mse}")
print(f"\t\tRMSE: {train_rmse}")
print(f"\t\tR²: {train_r2*100}")

# Evaluation Metrics for Testing Data
test_mae = mean_absolute_error(Y_test, Y_test_pred)
test_mse = mean_squared_error(Y_test, Y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(Y_test, Y_test_pred)

print("\n\tTesting Data Evaluation:")
print(f"\t\tMAE: {test_mae}")
print(f"\t\tMSE: {test_mse}")
print(f"\t\tRMSE: {test_rmse}")
print(f"\t\tR²: {test_r2*100}")

**Visualization Code**

In [ ]:
# Plotting Actual vs Predicted for Training Data
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.scatter(Y_train, Y_train_pred, color='blue', alpha=0.6)
plt.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTraining Data (XGBoost Regression)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Plotting Actual vs Predicted for Testing Data
plt.subplot(1, 2, 2)
plt.scatter(Y_test, Y_test_pred, color='green', alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTesting Data (XGBoost Regression)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Display the plots
plt.tight_layout()
plt.show()

**Robust Regression (RANSAC)**

In [ ]:
from sklearn.linear_model import RANSACRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
# Feature Scaling - RANSAC can be sensitive to the scale of features, so we scale them
scaler_X = StandardScaler()
scaler_Y = StandardScaler()

# Scaling the features
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Scaling the target variable
Y_train_scaled = scaler_Y.fit_transform(Y_train.values.reshape(-1, 1))
Y_test_scaled = scaler_Y.transform(Y_test.values.reshape(-1, 1))

In [ ]:
# Creating and training the RANSAC model
ransac = RANSACRegressor(random_state=2)
ransac.fit(X_train_scaled, Y_train_scaled.ravel())  # Use ravel to convert target to 1D

# Making predictions
Y_train_pred_scaled = ransac.predict(X_train_scaled)
Y_test_pred_scaled = ransac.predict(X_test_scaled)

# Inverse transform the scaled predictions to get them back to the original scale
Y_train_pred = scaler_Y.inverse_transform(Y_train_pred_scaled.reshape(-1, 1))
Y_test_pred = scaler_Y.inverse_transform(Y_test_pred_scaled.reshape(-1, 1))

In [ ]:
# Evaluation Metrics for Training Data
train_mae = mean_absolute_error(Y_train, Y_train_pred)
train_mse = mean_squared_error(Y_train, Y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(Y_train, Y_train_pred)


# Evaluation Metrics for Testing Data
test_mae = mean_absolute_error(Y_test, Y_test_pred)
test_mse = mean_squared_error(Y_test, Y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(Y_test, Y_test_pred)

print("Robust Regression (RANSAC) -\n")

# Printing the results
print("\tTraining Data Evaluation:")
print(f"\t\tMAE: {train_mae}")
print(f"\t\tMSE: {train_mse}")
print(f"\t\tRMSE: {train_rmse}")
print(f"\t\tR²: {-train_r2*100}")

print("\n\tTesting Data Evaluation:")
print(f"\t\tMAE: {test_mae}")
print(f"\t\tMSE: {test_mse}")
print(f"\t\tRMSE: {test_rmse}")
print(f"\t\tR²: {test_r2*100}")

In [ ]:
# Plotting Actual vs Predicted for Training Data
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.scatter(Y_train, Y_train_pred, color='blue', alpha=0.6)
plt.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTraining Data (Robust Regression (RANSAC))")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Plotting Actual vs Predicted for Testing Data
plt.subplot(1, 2, 2)
plt.scatter(Y_test, Y_test_pred, color='green', alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTesting Data (Robust Regression (RANSAC))")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Display the plots
plt.tight_layout()
plt.show()

 **Polynomial Regression**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import StandardScaler

# Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=2)

# Feature Scaling - Polynomial Regression can benefit from scaling the features
scaler_X = StandardScaler()
scaler_Y = StandardScaler()

# Scaling the features
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Scaling the target variable
Y_train_scaled = scaler_Y.fit_transform(Y_train.values.reshape(-1, 1))
Y_test_scaled = scaler_Y.transform(Y_test.values.reshape(-1, 1))

# Create Polynomial features (degree can be changed based on the need)
degree = 3  # You can tune this value
poly = PolynomialFeatures(degree=degree)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Create and train the linear regression model on the polynomial features
lin_reg = LinearRegression()
lin_reg.fit(X_train_poly, Y_train_scaled)

# Making predictions
Y_train_pred_scaled = lin_reg.predict(X_train_poly)
Y_test_pred_scaled = lin_reg.predict(X_test_poly)

# Inverse transform the scaled predictions to get them back to the original scale
Y_train_pred = scaler_Y.inverse_transform(Y_train_pred_scaled.reshape(-1, 1))
Y_test_pred = scaler_Y.inverse_transform(Y_test_pred_scaled.reshape(-1, 1))

# Evaluation Metrics for Training Data
train_mae = mean_absolute_error(Y_train, Y_train_pred)
train_mse = mean_squared_error(Y_train, Y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(Y_train, Y_train_pred)

# Evaluation Metrics for Testing Data
test_mae = mean_absolute_error(Y_test, Y_test_pred)
test_mse = mean_squared_error(Y_test, Y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(Y_test, Y_test_pred)

In [ ]:
print(f"Polynomial Regression (Degree {degree})-\n")
# Printing the results
print(f"\tTraining Data Evaluation:")
print(f"\t\tMAE: {train_mae}")
print(f"\t\tMSE: {train_mse}")
print(f"\t\tRMSE: {train_rmse}")
print(f"\t\tR²: {train_r2*100}")

print("\n\tTesting Data Evaluation:")
print(f"\t\tMAE: {test_mae}")
print(f"\t\tMSE: {test_mse}")
print(f"\t\tRMSE: {test_rmse}")
print(f"\t\tR²: {test_r2*100}")

In [ ]:
# Plotting Actual vs Predicted for Training Data
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.scatter(Y_train, Y_train_pred, color='blue', alpha=0.6)
plt.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTraining Data (Polynomial Regression)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Plotting Actual vs Predicted for Testing Data
plt.subplot(1, 2, 2)
plt.scatter(Y_test, Y_test_pred, color='green', alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTesting Data (Polinomial Regression)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Display the plots
plt.tight_layout()
plt.show()

**Bayesian Ridge Regression**

In [ ]:
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import StandardScaler

# Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=2)

# Feature Scaling - Bayesian Ridge Regression can benefit from scaling the features
scaler_X = StandardScaler()
scaler_Y = StandardScaler()

# Scaling the features
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Scaling the target variable
Y_train_scaled = scaler_Y.fit_transform(Y_train.values.reshape(-1, 1))
Y_test_scaled = scaler_Y.transform(Y_test.values.reshape(-1, 1))

# Create and train the Bayesian Ridge Regression model
bayesian_ridge = BayesianRidge()
bayesian_ridge.fit(X_train_scaled, Y_train_scaled.ravel())

# Making predictions
Y_train_pred_scaled = bayesian_ridge.predict(X_train_scaled)
Y_test_pred_scaled = bayesian_ridge.predict(X_test_scaled)

# Inverse transform the scaled predictions to get them back to the original scale
Y_train_pred = scaler_Y.inverse_transform(Y_train_pred_scaled.reshape(-1, 1))
Y_test_pred = scaler_Y.inverse_transform(Y_test_pred_scaled.reshape(-1, 1))

# Evaluation Metrics for Training Data
train_mae = mean_absolute_error(Y_train, Y_train_pred)
train_mse = mean_squared_error(Y_train, Y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(Y_train, Y_train_pred)

# Evaluation Metrics for Testing Data
test_mae = mean_absolute_error(Y_test, Y_test_pred)
test_mse = mean_squared_error(Y_test, Y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(Y_test, Y_test_pred)


print("Bayesian Ridge Regression-\n")
# Printing the results
print("\tTraining Data Evaluation:")
print(f"\t\tMAE: {train_mae}")
print(f"\t\tMSE: {train_mse}")
print(f"\t\tRMSE: {train_rmse}")
print(f"\t\tR²: {train_r2*100}")

print("\n\tTesting Data Evaluation:")
print(f"\t\tMAE: {test_mae}")
print(f"\t\tMSE: {test_mse}")
print(f"\t\tRMSE: {test_rmse}")
print(f"\t\tR²: {test_r2*100}")

In [ ]:
# Plotting Actual vs Predicted for Training Data
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.scatter(Y_train, Y_train_pred, color='blue', alpha=0.6)
plt.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTraining Data (Bayesian Ridge Regression)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Plotting Actual vs Predicted for Testing Data
plt.subplot(1, 2, 2)
plt.scatter(Y_test, Y_test_pred, color='green', alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--', linewidth=2)
plt.title("Actual vs Predicted - \nTesting Data (Bayesian Ridge Regression)")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()

# Display the plots
plt.tight_layout()
plt.show()

**Determine the Best Model**

In [ ]:
import numpy as np
from scipy import stats

In [ ]:
# Adding the intercept term
x = np.column_stack((np.ones(X.shape[0]), X))

In [ ]:
# Calculate the coefficients (beta)
beta = np.linalg.inv(x.T @ x) @ x.T @ Y

In [ ]:
# Calculate the predictions
y_pred = x @ beta

In [ ]:
# Calculate residuals
residuals = Y - y_pred


In [ ]:
# Calculate the variance and standard error
sigma_squared = np.sum(residuals**2) / (len(Y) - 2)
standard_error = np.sqrt(np.diagonal(sigma_squared * np.linalg.inv(x.T @ x)))

In [ ]:
# Calculate t-statistics
t_stats = beta / standard_error


In [ ]:
# Calculate p-values
p_values = [2 * (1 - stats.t.cdf(np.abs(t), len(Y) - 2)) for t in t_stats]

In [ ]:
# Print the results
print("Coefficients:", beta)
print("P-values:", p_values)

 **P_ Value for Random forest Regression.**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
import pandas as pd

# Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Fitting the Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, Y_train)

# Permutation importance
perm_importance = permutation_importance(rf, X_test, Y_test, n_repeats=30, random_state=42)

# Display feature importance and approximate p-values
feature_importances = perm_importance.importances_mean
p_values = [1.0 if imp <= 0 else (sum(perm_importance.importances[i] <= 0) / len(perm_importance.importances[i]))
            for i, imp in enumerate(perm_importance.importances_mean)]

results = pd.DataFrame({
    "Feature": X.columns,
    "Importance": feature_importances,
    "P-value Approximation": p_values
}).sort_values(by="Importance", ascending=False)

print(results)

 **P_ Value for XGboost Regression.**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
import pandas as pd

# Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Fitting the Random Forest model
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=1)
xgb.fit(X_train, Y_train)

# Permutation importance
perm_importance = permutation_importance(xgb, X_test, Y_test, n_repeats=30, random_state=42)

# Display feature importance and approximate p-values
feature_importances = perm_importance.importances_mean
p_values = [1.0 if imp <= 0 else (sum(perm_importance.importances[i] <= 0) / len(perm_importance.importances[i]))
            for i, imp in enumerate(perm_importance.importances_mean)]

results = pd.DataFrame({
    "Feature": X.columns,
    "Importance": feature_importances,
    "P-value Approximation": p_values
}).sort_values(by="Importance", ascending=False)

print(results)


**Actual vs Predicted Values**

In [ ]:
# Actual vs Predicted Values
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import matplotlib.pyplot as plt


# Step 1: Scale the target variable (optional if scaling needed)
scaler = MinMaxScaler()
Y_scaled = scaler.fit_transform(Y.values.reshape(-1, 1))

# Step 2: Split the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_scaled, test_size=0.2, random_state=42)

# Step 3: Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, Y_train.ravel())

# Step 4: Train XGBoost model
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, Y_train.ravel())

# Step 5: Make predictions
Y_pred_rf = rf_model.predict(X_test)
Y_pred_xgb = xgb_model.predict(X_test)

# Step 6: Inverse transform predictions (if scaled)
Y_test_original = scaler.inverse_transform(Y_test)
Y_pred_rf_original = scaler.inverse_transform(Y_pred_rf.reshape(-1, 1))
Y_pred_xgb_original = scaler.inverse_transform(Y_pred_xgb.reshape(-1, 1))

# Step 7: Plot Actual vs Predicted values
plt.figure(figsize=(12, 8))
plt.plot(range(len(Y_test_original)), Y_test_original, label='Actual Values', color='blue', alpha=0.7)
plt.plot(range(len(Y_test_original)), Y_pred_rf_original, label='Random Forest Predictions', color='orange', linestyle='--')
plt.plot(range(len(Y_test_original)), Y_pred_xgb_original, label='XGBoost Predictions', color='green', linestyle='--')
plt.title('Actual vs Predicted Values')
plt.xlabel('Index')
plt.ylabel('Target Values')
plt.legend()
plt.grid(True)
plt.show()

**Ensemble**

In [ ]:
# To create an ensemble of the three regression models (`RandomForestRegressor`, `XGBRegressor`, and `RANSACRegressor`), we can use a stacking technique. Stacking involves training a meta-model that combines the predictions from the base models to improve the overall performance.

# Here is the code to implement the ensemble using stacking:

# ```python
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression  # Meta-model
from sklearn.pipeline import Pipeline

# Random Forest model
rf = RandomForestRegressor()

# XGBoost model
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=1)

# RANSAC model (with scaling pipeline)
ransac_pipeline = Pipeline([
    ('scaler_X', StandardScaler()),
    ('ransac', RANSACRegressor(random_state=2))
])

# Creating the Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=[
        ('random_forest', rf),
        ('xgboost', xgb),
        ('ransac', ransac_pipeline)
    ],
    final_estimator=LinearRegression(),  # Meta-model
    passthrough=False  # Use base model predictions as inputs to the meta-model
)

# Train the Stacking Regressor
stacking_regressor.fit(X_train, Y_train)

# Making predictions
Y_train_pred = stacking_regressor.predict(X_train)
Y_test_pred = stacking_regressor.predict(X_test)

# Evaluation Metrics for Training Data
train_mae = mean_absolute_error(Y_train, Y_train_pred)
train_mse = mean_squared_error(Y_train, Y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(Y_train, Y_train_pred)

print("Stacking Regression -\n")
# Printing the results
print("\tTraining Data Evaluation:")
print(f"\t\tMAE: {train_mae}")
print(f"\t\tMSE: {train_mse}")
print(f"\t\tRMSE: {train_rmse}")
print(f"\t\tR²: {train_r2*100}")

# Evaluation Metrics for Testing Data
test_mae = mean_absolute_error(Y_test, Y_test_pred)
test_mse = mean_squared_error(Y_test, Y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(Y_test, Y_test_pred)

print("\n\tTesting Data Evaluation:")
print(f"\t\tMAE: {test_mae}")
print(f"\t\tMSE: {test_mse}")
print(f"\t\tRMSE: {test_rmse}")
print(f"\t\tR²: {test_r2*100}")




In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Evaluation metrics for visualization
metrics = ['MAE', 'MSE', 'RMSE', 'R²']
train_scores = [train_mae, train_mse, train_rmse, train_r2 * 100]  # R² as percentage
test_scores = [test_mae, test_mse, test_rmse, test_r2 * 100]       # R² as percentage

x = np.arange(len(metrics))  # Number of metrics

# Plotting
plt.figure(figsize=(10, 6))
bar_width = 0.35

# Bar plots for Training and Testing scores
plt.bar(x - bar_width/2, train_scores, width=bar_width, color='skyblue', label='Training')
plt.bar(x + bar_width/2, test_scores, width=bar_width, color='salmon', label='Testing')

# Adding annotations to the bars
for i in range(len(metrics)):
    plt.text(x[i] - bar_width/2, train_scores[i] + 0.5, f"{train_scores[i]:.2f}", ha='center', va='bottom', fontsize=10)
    plt.text(x[i] + bar_width/2, test_scores[i] + 0.5, f"{test_scores[i]:.2f}", ha='center', va='bottom', fontsize=10)

# Customizing the plot
plt.xticks(x, metrics, fontsize=12)
plt.xlabel('Metrics', fontsize=14)
plt.ylabel('Scores', fontsize=14)
plt.title('Performance of Stacking Regressor on Training and Testing Data', fontsize=16)
plt.legend(fontsize=12)
plt.tight_layout()

# Display the plot
plt.show()
